2023/12
## skku DataMining project : 
## 서울 대중교통 혼잡도 및 중심성 그래프분석(Seoul_transportation_GraphMining)
contributer : 김민, 김효원, 이수정, 조병웅  
instruction :   
블로거 GD park manger님의 대중교통 분석 연구를 바탕으로, 서울시 대중교통 네트워크의 중심성을 살펴보고,
사용목적 대비 유동인구 수라는 혼잡도 개념을 추가하여 새로 발전된 인사이트를 제공함을 목적으로 함.
참고한 분석 연구의 주소 -> https://m.blog.naver.com/gdpresent/221147909255

## graph_congestion 파일 정보
contributer : 이수정
전처리1에서 완성된 기본 그래프 정보를 바탕으로 지하철 인구 이동 od 데이터를 사용해 혼잡도를 구하는 코드

## 라이브러리 및 데이터 로딩

In [1]:
import pandas as pd
import networkx as nx

In [ ]:
node_subway = pd.read_csv('../데이터/전처리1_생성데이터/node_subway.csv', encoding='cp949')
node_bus = pd.read_csv('../데이터/전처리1_생성데이터/node_bus.csv', encoding='cp949')
edge_subway = pd.read_csv('../데이터/전처리1_생성데이터/edge_subway.csv', encoding='cp949')
edge_bus = pd.read_csv('../데이터/전처리1_생성데이터/edge_bus.csv', encoding='cp949')

In [ ]:
node_subway['type'] = 'subway'
node_bus['type'] = 'bus'
node = pd.concat([node_subway, node_bus], ignore_index=True)
node.drop_duplicates(subset=['title'], inplace=True) # title 기준 중복 제거
node.sort_values(by=['ARS_ID'], inplace=True)
node

,title,ARS_ID,name,X,Y,type
0,1_4.19민주묘지역,1,4.19민주묘지역,37.649527,127.013689,subway
1,2_가능역,2,가능역,37.748393,127.044275,subway
2,3_가락시장역,3,가락시장역,37.493092,127.118258,subway
3,4_가산디지털단지역,4,가산디지털단지역,37.480389,126.882644,subway
4,5_가양역,5,가양역,37.561429,126.854402,subway
...,...,...,...,...,...,...
896,68270_광사1통마을회관,68270,광사1통마을회관,37.788065,127.069836,bus
920,68271_광사1통마을회관,68271,광사1통마을회관,37.788093,127.070235,bus
870,68290_덕정주공6단지,68290,덕정주공6단지,37.837017,127.073694,bus
885,68351_삼숭교차로,68351,삼숭교차로,37.810362,127.103922,bus


In [ ]:
edge_add = edge_subway.copy()
edge_add['source'], edge_add['target'] = edge_subway['target'], edge_subway['source']
edge_add

,LaneNo,source,target
0,1,89_덕계역,90_덕정역
1,1,272_용산역,63_남영역
2,1,68_노량진역,272_용산역
3,1,84_대방역,68_노량진역
4,1,210_신길역,84_대방역
...,...,...,...
431,113,6_가오리역,348_화계역
432,113,1_4.19민주묘지역,6_가오리역
433,113,194_솔밭공원역,1_4.19민주묘지역
434,113,153_북한산우이역,194_솔밭공원역


In [ ]:
edge_subway['type'] = 'subway'
edge_add['type'] = 'subway'
edge_bus['type'] = 'bus'

edge = pd.concat([edge_subway, edge_add, edge_bus], ignore_index=True)
edge

,LaneNo,source,target,type
0,1,90_덕정역,89_덕계역,subway
1,1,63_남영역,272_용산역,subway
2,1,272_용산역,68_노량진역,subway
3,1,68_노량진역,84_대방역,subway
4,1,84_대방역,210_신길역,subway
...,...,...,...,...
14272,775,63240_교육지원청,63242_파주시청,bus
14273,775,63242_파주시청,63233_장안아파트,bus
14274,775,63233_장안아파트,63201_금촌사거리,bus
14275,775,63201_금촌사거리,63213_장안미래아파트,bus


In [ ]:
edge["source"]=edge['source'].str.replace(" ","")
edge["target"]=edge['target'].str.replace(" ","")
node["title"]=node['title'].str.replace(" ","")

### 혼잡지수_버스

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/데이터마이닝팀플/데이터/대중교통_월간통계자료_2023.08월/노선별/노선별OD/노선별OD_20230815.csv",encoding="cp949")
df.head()
df = df.dropna()

<ipython-input-8-77ad9eb49307>:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/drive/MyDrive/데이터마이닝팀플/데이터/대중교통_월간통계자료_2023.08월/노선별/노선별OD/노선별OD_20230815.csv",encoding="cp949")


,기준일자,노선명,승차_정류장ARS,승차_정류장명,하차_정류장ARS,하차_정류장명,승차_정류장순번,하차_정류장순번,승객수
0,20230815,0017,3689.0,청암자이아파트,3247.0,용문시장,1.0,9.0,1
1,20230815,0017,3689.0,청암자이아파트,3255.0,용산역,1.0,12.0,9
2,20230815,0017,3689.0,청암자이아파트,3298.0,청암동강변삼성아파트,1.0,2.0,1
3,20230815,0017,3689.0,청암자이아파트,3310.0,남이장군사당,1.0,7.0,2
4,20230815,0017,3689.0,청암자이아파트,3312.0,새마을금고,1.0,8.0,3


In [ ]:
# 간선버스 노선 추출 (노선번호 3자리 숫자)

df = df[df['노선명'].str.match(r'^\d{3}$|^\d{3}[A-Z]{1}$',na=False)]
df = df.sort_values(by=['노선명']).reset_index(drop=True) # 노선명, 정렬

In [ ]:
df.head()

,기준일자,노선명,승차_정류장ARS,승차_정류장명,하차_정류장ARS,하차_정류장명,승차_정류장순번,하차_정류장순번,승객수
0,20230815,100,11428.0,한성여객종점,11373.0,중계역,1.0,4.0,1
1,20230815,100,2142.0,롯데영프라자,11433.0,대진고등학교,78.0,113.0,7
2,20230815,100,2142.0,롯데영프라자,11429.0,한성여객종점,78.0,112.0,2
3,20230815,100,2142.0,롯데영프라자,11425.0,하계미성아파트.을지병원,78.0,111.0,1
4,20230815,100,2142.0,롯데영프라자,11375.0,중계건영2차아파트,78.0,119.0,2


In [ ]:
# float형을 int형으로 변환
edge["weight"]=0
df["승차_정류장ARS"] = df["승차_정류장ARS"].astype(int)
df["하차_정류장ARS"] = df["하차_정류장ARS"].astype(int)

In [ ]:
# 가상 정류소(정차X) 제거
print('가상 정류소 제거 전:', len(df))
fake = df[df['승차_정류장명'].str.contains('가상')]
df.drop(fake.index, axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)
fake = df[df['하차_정류장명'].str.contains('가상')]
df.drop(fake.index, axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)
print('가상 정류소 제거 후:', len(df))
df

가상 정류소 제거 전: 110196
가상 정류소 제거 후: 109799


,기준일자,노선명,승차_정류장ARS,승차_정류장명,하차_정류장ARS,하차_정류장명,승차_정류장순번,하차_정류장순번,승객수
0,20230815,100,11428,한성여객종점,11373,중계역,1.0,4.0,1
1,20230815,100,2142,롯데영프라자,11433,대진고등학교,78.0,113.0,7
2,20230815,100,2142,롯데영프라자,11429,한성여객종점,78.0,112.0,2
3,20230815,100,2142,롯데영프라자,11425,하계미성아파트.을지병원,78.0,111.0,1
4,20230815,100,2142,롯데영프라자,11375,중계건영2차아파트,78.0,119.0,2
...,...,...,...,...,...,...,...,...,...
109794,20230815,774,35207,벽제역,35241,고양동사거리,29.0,36.0,2
109795,20230815,774,35207,벽제역,35210,국군고양병원,29.0,40.0,1
109796,20230815,774,35207,벽제역,35192,푸른마을10단지.대명교회앞,29.0,38.0,1
109797,20230815,774,35217,선유동입구,35226,신성빌라.동익아파트,31.0,37.0,1


In [ ]:
# 괄호 내용 제거 함수
def delWord(station):
    if '(' in station:
        try:
            idx1 = station.index('(')
            idx2 = station.index(')')
            return station[:idx1] + station[idx2+1:]
        except ValueError as e:
            pass
    return station

In [ ]:
# '역' 포함 버스 정류소 추출
df['승차_정류장명'] = df['승차_정류장명'].map(delWord)
df['하차_정류장명'] = df['하차_정류장명'].map(delWord)
condition = df['승차_정류장명'].str.contains('역') & ~df['승차_정류장명'].str.contains('역사문화|역사박물관|역사재단|한국지역|기차역')
df_trans1 = df[condition]
df_trans1['승차_정류장명'] = df_trans1['승차_정류장명'].map(delWord) # 정류소명 내 괄호 제거
df_trans1

condition2 = df['하차_정류장명'].str.contains('역') & ~df['하차_정류장명'].str.contains('역사문화|역사박물관|역사재단|한국지역|기차역')

df_trans = df[(condition)| (condition2)]
df_trans.reset_index(drop=True, inplace=True)
df_trans_out =df[(~condition)& (~condition2)]

df_trans.reset_index(drop=True, inplace=True)

#역이 포함되지 않은 것만
df_trans_out.reset_index(drop=True, inplace=True)

<ipython-input-15-569569210ee3>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trans1['승차_정류장명'] = df_trans1['승차_정류장명'].map(delWord) # 정류소명 내 괄호 제거


In [ ]:
# 버스 정류소별 ARS ID
busDict1 = df_trans.set_index('승차_정류장명').to_dict()['승차_정류장ARS']
print('버스:', busDict1)
busDict2 = df_trans.set_index('하차_정류장명').to_dict()['하차_정류장ARS']
print('버스:', busDict2)

# 지하철 ARS ID
subway = pd.read_csv('./노드엣지(8월)/node_subway.csv', encoding='cp949')
subDict = subway.set_index('name').to_dict()['ARS_ID']
print('지하철:', subDict)

버스: {'한성여객종점': 11428, '롯데영프라자': 2142, '을지로입구.로얄호텔': 2156, '남대문시장앞.이회영활동터': 2219, '서울역버스환승센터.강우규의거터': 2007, '을지로4가.베스트웨스턴국도': 2163, '을지로5가.중부시장': 2166, '을지로6가.국립중앙의료원': 2169, '을지로2가.파인에비뉴': 2159, '을지로3가': 2161, '이촌동한가람아파트.이촌역': 3217, '신용산초등학교': 3219, '한강대교북단.LG유플러스': 3002, '이촌동두산위브트레지움.한강대우아파트': 3215, '이촌동현대아파트': 3223, '서빙고신동아아파트': 3226, '중경고등학교. 이촌동 강촌아파트': 3221, '숙대입구역': 3010, '삼각지역': 3008, '갈월동': 3012, '신용산역': 3004, '용산우체국': 3006, '창문여고': 8102, '돈암사거리.성신여대입구': 8008, '길음뉴타운': 8004, '중계역': 11366, '중계건영2차아파트': 11375, '상명초등학교': 11377, '롯데마트': 11358, '하계현대아파트': 11442, '서울온천': 11435, '종로5가': 1212, '방산시장.훈련원종합체육관': 2167, '종로5가.효제초등학교': 1205, '퇴계로6가': 2206, '광희동': 2171, '을지로5가.훈련원종합체육관': 2168, '혜화역.동성중고': 1229, '혜화역.마로니에공원': 1220, '방송통신대.이화장': 1219, '종로5가.효제동.김상옥의거터': 1204, '인덕대학': 11311, '월계주공108동앞롯데캐슬루나아파트': 11310, '하계극동아파트': 11452, '하계우성아파트': 11454, '월계보건지소': 11339, '송중동한일유엔아이': 9188, '숭곡초등학교입구': 9190, '창문여고앞': 9189, '오현초등학교': 9186, '북서울꿈의숲': 8199, '꿈의숲주차장입구': 9312, '목련아파트상가': 11380, '중계3

In [ ]:
busDict = busDict1
temp = list(busDict2.keys())
for i in range(len(busDict2)):
    if temp[i] in busDict:
        busDict[temp[i]] = busDict2[temp[i]]
    else:
        busDict[temp[i]] = busDict2[temp[i]]


In [ ]:
# 지하철 환승 가능 버스 정류소에 지하철역 ARS ID 부여하는 함수
def getID(x):
    if '연신내역' in x: # 신내역/연신내역 구분
        return subDict['연신내역']
    for sub in subway.name:
        if sub in x:
            return subDict[sub]
#    return busDict[x]

In [ ]:

df_trans['newID_승차'] = df_trans['승차_정류장명'].map(getID)
df_trans['newID_하차'] = df_trans['하차_정류장명'].map(getID)
trans = df_trans

trans.head(8)

<ipython-input-19-79c2f73c6e49>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trans['newID_승차'] = df_trans['승차_정류장명'].map(getID)
<ipython-input-19-79c2f73c6e49>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trans['newID_하차'] = df_trans['하차_정류장명'].map(getID)


,기준일자,노선명,승차_정류장ARS,승차_정류장명,하차_정류장ARS,하차_정류장명,승차_정류장순번,하차_정류장순번,승객수,newID_승차,newID_하차
0,20230815,100,11428,한성여객종점,11373,중계역,1.0,4.0,1,NaN,308.0
1,20230815,100,2142,롯데영프라자,1229,혜화역.동성중고,78.0,94.0,4,NaN,345.0
2,20230815,100,2156,을지로입구.로얄호텔,1220,혜화역.마로니에공원,79.0,93.0,2,NaN,345.0
3,20230815,100,2142,롯데영프라자,1220,혜화역.마로니에공원,78.0,93.0,4,NaN,345.0
4,20230815,100,2219,남대문시장앞.이회영활동터,1229,혜화역.동성중고,77.0,94.0,1,NaN,345.0
5,20230815,100,2219,남대문시장앞.이회영활동터,1220,혜화역.마로니에공원,77.0,93.0,1,NaN,345.0
6,20230815,100,2007,서울역버스환승센터.강우규의거터,11455,하계장미아파트,76.0,110.0,16,181.0,NaN
7,20230815,100,2007,서울역버스환승센터.강우규의거터,11453,하계시영아파트,76.0,109.0,2,181.0,NaN


In [ ]:
import numpy as np
for i in range(len(trans)):
    if np.isnan(trans["newID_승차"].iloc[i]):
        trans["newID_승차"].iloc[i] = trans["승차_정류장ARS"].iloc[i]
    if np.isnan(trans["newID_하차"].iloc[i]):
        trans["newID_하차"].iloc[i] = trans["하차_정류장ARS"].iloc[i]

<ipython-input-125-c777799d730a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trans["newID_승차"].iloc[i] = trans["승차_정류장ARS"].iloc[i]
<ipython-input-125-c777799d730a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trans["newID_하차"].iloc[i] = trans["하차_정류장ARS"].iloc[i]


In [ ]:
bus_info = pd.read_excel("/content/drive/MyDrive/데이터마이닝팀플/그래프코드(231126)/서울시버스노선기본정보(20230914) (2).xlsx")
#bus_info["노선_ID"] = bus_info["노선_ID"].astype(str)

In [ ]:
bus_info

,업체명,노선번호,유형,기점명,종점명,인가대수,배차간격,거리,운행시간,최소배차,최대배차,첫차시간,막차시간
0,보광교통,0017,지선,청암동,이촌동,10,12,12.0,60.0,8,13,05:15,23:30
1,북부운수,01,순환,예장주차장,예장주차장,14,8,16.0,60.0,6,10,06:30,23:00
2,대원여객,0411,지선,용산차고지,AT센터.양재꽃시장,21,14,44.0,220.0,12,19,04:20,22:30
3,한성여객,100,간선,하계동,용산구청,32,10,57.0,231.0,8,13,04:00,22:30
4,한성운수,101,간선,우이동,서소문,24,10,38.0,170.0,5,14,04:00,23:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
697,약수교통,종로13,마을,평창동주민센터,부암슈퍼,4,15,8.0,50.0,10,20,05:50,22:30
698,금창운수 월계점,중랑01,마을,중화1동동아약국,신이문역,2,25,4.0,24.0,25,25,06:00,23:50
699,금창운수,중랑01,마을,중화1동동아약국,신이문역,2,25,4.0,24.0,25,25,06:00,23:50
700,중랑운수,중랑02,마을,진로아파트,한신아파트,7,8,7.0,40.0,5,12,06:00,23:15


In [ ]:
df_trans_out['newID_승차'] = df_trans_out['승차_정류장ARS']
df_trans_out['newID_하차'] = df_trans_out['하차_정류장ARS']
df_trans_out.reset_index(drop=True,inplace=True)

<ipython-input-128-45e5744649ca>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trans_out['newID_승차'] = df_trans_out['승차_정류장ARS']
<ipython-input-128-45e5744649ca>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trans_out['newID_하차'] = df_trans_out['하차_정류장ARS']


In [ ]:
#지하철 관련과 비관련 결합
trans1 = pd.concat([trans,df_trans_out])
trans = trans1.drop_duplicates()

In [ ]:
#서울시버스노선기본정보에 없는 노선 삭제하기 위함
trans.reset_index(drop=True,inplace=True)
dictionary ={}
no_list_ =[]
for k in list(trans["노선명"].unique()):
    print(k)
    for i in range(len(bus_info)):
        try:
            last_h = int(bus_info[bus_info["노선번호"]==k]["막차시간"].values[0].split(":")[0])
            first_h = int(bus_info[bus_info["노선번호"]==k]["첫차시간"].values[0].split(":")[0])
            if last_h ==0:
                last_h = 24

            last_m = int(bus_info[bus_info["노선번호"]==k]["막차시간"].values[0].split(":")[1])
            first_m = int(bus_info[bus_info["노선번호"]==k]["첫차시간"].values[0].split(":")[1])



            dictionary[k]=((last_h - first_h)*60 +(last_m - first_m) )/ (bus_info[bus_info["노선번호"]==k]["배차간격"].values[0])
        except IndexError as e:
            print("No:",k )
            no_list_.append(k)
            break


100
101
102
103
104
105
106
107
109
111
No: 111
120
121
No: 121
130
No: 130
140
141
142
143
144
No: 144
145
146
147
148
150
472
500
No: 500
501
502
503
504
505
506
507
No: 507
540
541
542
571
600
No: 600
601
602
603
604
605
No: 605
606
640
641
643
650
651
No: 651
652
653
654
No: 654
660
661
662
672
No: 672
673
674
700
701
703
704
705
708
710
720
721
740
741
742
750A
750B
No: 750B
752
753
760
761
771
773
774


In [ ]:
 no_list_ =['111',
  '121',
  '130',
  '144',
  '500',
  '507',
  '600',
  '605',
  '651',
  '654',
  '672',
 '750B']

In [ ]:
# 혼잡지수를 만들기 위해 운행횟수 추정
trans = trans[trans['newID_승차'] != trans['newID_하차']]
print(len(trans))
trans["운행횟수"] =0
for i in range(len(trans)):
    for key, value in dictionary.items():
        if key == trans["노선명"].iloc[i]:
            trans["운행횟수"].iloc[i] =value
            print(i)
            break


106096
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275

<ipython-input-132-ea24d5d15e5d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trans["운행횟수"] =0
<ipython-input-132-ea24d5d15e5d>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trans["운행횟수"].iloc[i] =value


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
101096
101097
101098
101099
101100
101101
101102
101103
101104
101105
101106
101107
101108
101109
101110
101111
101112
101113
101114
101115
101116
101117
101118
101119
101120
101121
101122
101123
101124
101125
101126
101127
101128
101129
101130
101131
101132
101133
101134
101135
101136
101137
101138
101139
101140
101141
101142
101143
101144
101145
101146
101147
101148
101149
101150
101151
101152
101153
101154
101155
101156
101157
101158
101159
101160
101161
101162
101163
101164
101165
101166
101167
101168
101169
101170
101171
101172
101173
101174
101175
101176
101177
101178
101179
101180
101181
101182
101183
101184
101185
101186
101187
101188
101189
101190
101191
101192
101193
101194
101195
101196
101197
101198
101199
101200
101201
101202
101203
101204
101205
101206
101207
101208
101209
101210
101211
101212
101213
101214
101215
101216
101217
101218
101219
101220
101221
101222
101223
101224
101225
101226
101227
101228
101229
101230
101231
101232
10123

In [ ]:
# 혼잡지수 생성
trans["혼잡지수"]= 0
for i in range(len(trans)):
    trans["혼잡지수"].iloc[i] = trans["승객수"].iloc[i]/(trans["운행횟수"].iloc[i]*28)

<ipython-input-133-58770ab34614>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trans["혼잡지수"].iloc[i] = trans["승객수"].iloc[i]/(trans["운행횟수"].iloc[i]*28)
<ipython-input-133-58770ab34614>:3: RuntimeWarning: divide by zero encountered in double_scalars
  trans["혼잡지수"].iloc[i] = trans["승객수"].iloc[i]/(trans["운행횟수"].iloc[i]*28)


In [ ]:
#float형을 int형으로 변환
trans["newID_승차"] = trans["newID_승차"].astype(int)
trans["newID_하차"] = trans["newID_하차"].astype(int)
trans

,기준일자,노선명,승차_정류장ARS,승차_정류장명,하차_정류장ARS,하차_정류장명,승차_정류장순번,하차_정류장순번,승객수,newID_승차,newID_하차,운행횟수,혼잡지수
0,20230815,100,11428,한성여객종점,11373,중계역,1.0,4.0,1,11428,308,111.0,0.000322
1,20230815,100,2142,롯데영프라자,1229,혜화역.동성중고,78.0,94.0,4,2142,345,111.0,0.001287
2,20230815,100,2156,을지로입구.로얄호텔,1220,혜화역.마로니에공원,79.0,93.0,2,2156,345,111.0,0.000644
3,20230815,100,2142,롯데영프라자,1220,혜화역.마로니에공원,78.0,93.0,4,2142,345,111.0,0.001287
4,20230815,100,2219,남대문시장앞.이회영활동터,1229,혜화역.동성중고,77.0,94.0,1,2219,345,111.0,0.000322
...,...,...,...,...,...,...,...,...,...,...,...,...,...
109794,20230815,774,35217,선유동입구,35181,고양동삼거리,31.0,39.0,1,35217,35181,48.4,0.000738
109795,20230815,774,35217,선유동입구,31612,광탄행정복지센터,31.0,56.0,1,35217,31612,48.4,0.000738
109796,20230815,774,35217,선유동입구,31584,용미2리,31.0,47.0,1,35217,31584,48.4,0.000738
109797,20230815,774,35187,부대앞,35605,고양현대아파트,30.0,34.0,1,35187,35605,48.4,0.000738


In [ ]:
# 고유한 id를 만들기 위한 거으로 ARS_ID와 정류장명을 결합
idList, nameList = list(trans.newID_승차), list(trans.승차_정류장명)
title = ['%d_%s'%(idList[i], nameList[i]) for i in range(len(trans))]
trans['승차_title'] = title

idList, nameList = list(trans.newID_하차), list(trans.하차_정류장명)
title = ['%d_%s'%(idList[i], nameList[i]) for i in range(len(trans))]
trans['하차_title'] = title

trans

,기준일자,노선명,승차_정류장ARS,승차_정류장명,하차_정류장ARS,하차_정류장명,승차_정류장순번,하차_정류장순번,승객수,newID_승차,newID_하차,운행횟수,혼잡지수,승차_title,하차_title
0,20230815,100,11428,한성여객종점,11373,중계역,1.0,4.0,1,11428,308,111.0,0.000322,11428_한성여객종점,308_중계역
1,20230815,100,2142,롯데영프라자,1229,혜화역.동성중고,78.0,94.0,4,2142,345,111.0,0.001287,2142_롯데영프라자,345_혜화역.동성중고
2,20230815,100,2156,을지로입구.로얄호텔,1220,혜화역.마로니에공원,79.0,93.0,2,2156,345,111.0,0.000644,2156_을지로입구.로얄호텔,345_혜화역.마로니에공원
3,20230815,100,2142,롯데영프라자,1220,혜화역.마로니에공원,78.0,93.0,4,2142,345,111.0,0.001287,2142_롯데영프라자,345_혜화역.마로니에공원
4,20230815,100,2219,남대문시장앞.이회영활동터,1229,혜화역.동성중고,77.0,94.0,1,2219,345,111.0,0.000322,2219_남대문시장앞.이회영활동터,345_혜화역.동성중고
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109794,20230815,774,35217,선유동입구,35181,고양동삼거리,31.0,39.0,1,35217,35181,48.4,0.000738,35217_선유동입구,35181_고양동삼거리
109795,20230815,774,35217,선유동입구,31612,광탄행정복지센터,31.0,56.0,1,35217,31612,48.4,0.000738,35217_선유동입구,31612_광탄행정복지센터
109796,20230815,774,35217,선유동입구,31584,용미2리,31.0,47.0,1,35217,31584,48.4,0.000738,35217_선유동입구,31584_용미2리
109797,20230815,774,35187,부대앞,35605,고양현대아파트,30.0,34.0,1,35187,35605,48.4,0.000738,35187_부대앞,35605_고양현대아파트


In [ ]:
# 중간 저장
trans.to_csv("중간.csv",index=False, encoding="cp949")

In [ ]:
trans= pd.read_csv("중간.csv",encoding="cp949")
trans.head(7)

,기준일자,노선명,승차_정류장ARS,승차_정류장명,하차_정류장ARS,하차_정류장명,승차_정류장순번,하차_정류장순번,승객수,newID_승차,newID_하차,운행횟수,혼잡지수,승차_title,하차_title
0,20230815,100,11428,한성여객종점,11373,중계역,1.0,4.0,1,11428,308,111.0,0.000322,11428_한성여객종점,308_중계역
1,20230815,100,2142,롯데영프라자,1229,혜화역.동성중고,78.0,94.0,4,2142,345,111.0,0.001287,2142_롯데영프라자,345_혜화역.동성중고
2,20230815,100,2156,을지로입구.로얄호텔,1220,혜화역.마로니에공원,79.0,93.0,2,2156,345,111.0,0.000644,2156_을지로입구.로얄호텔,345_혜화역.마로니에공원
3,20230815,100,2142,롯데영프라자,1220,혜화역.마로니에공원,78.0,93.0,4,2142,345,111.0,0.001287,2142_롯데영프라자,345_혜화역.마로니에공원
4,20230815,100,2219,남대문시장앞.이회영활동터,1229,혜화역.동성중고,77.0,94.0,1,2219,345,111.0,0.000322,2219_남대문시장앞.이회영활동터,345_혜화역.동성중고
5,20230815,100,2219,남대문시장앞.이회영활동터,1220,혜화역.마로니에공원,77.0,93.0,1,2219,345,111.0,0.000322,2219_남대문시장앞.이회영활동터,345_혜화역.마로니에공원
6,20230815,100,2007,서울역버스환승센터.강우규의거터,11455,하계장미아파트,76.0,110.0,16,181,11455,111.0,0.005148,181_서울역버스환승센터.강우규의거터,11455_하계장미아파트


In [ ]:
# edge 노선 제외 데이터프레임 추출
edge = edge[~(edge["LaneNo"].isin(no_list_))].reset_index(drop=True)

In [ ]:
# 노선 지워도 혼자서 따로 노는 노드 존재 (원래는 노드 4890 개)
confirm1 = list(edge["source"].values)
confirm2 = list(edge["target"].values)
confirm1.extend(confirm2)
confirm1=list(set(confirm1))
len(set(confirm1))

4770

In [ ]:
# 혼자 따로 노는 노드 삭제
node =node[(node["title"].isin(confirm1))].reset_index(drop=True)
node

,title,ARS_ID,name,X,Y,type
0,1_4.19민주묘지역,1,4.19민주묘지역,37.649527,127.013689,subway
1,2_가능역,2,가능역,37.748393,127.044275,subway
2,3_가락시장역,3,가락시장역,37.493092,127.118258,subway
3,4_가산디지털단지역,4,가산디지털단지역,37.480389,126.882644,subway
4,5_가양역,5,가양역,37.561429,126.854402,subway
...,...,...,...,...,...,...
4765,68270_광사1통마을회관,68270,광사1통마을회관,37.788065,127.069836,bus
4766,68271_광사1통마을회관,68271,광사1통마을회관,37.788093,127.070235,bus
4767,68290_덕정주공6단지,68290,덕정주공6단지,37.837017,127.073694,bus
4768,68351_삼숭교차로,68351,삼숭교차로,37.810362,127.103922,bus


In [ ]:
#서울시버스노선기본정보에 없는 노선 삭제
trans = trans[~(trans["노선명"].isin(no_list_))].reset_index(drop=True)

In [ ]:
graph = nx.DiGraph()
graph.add_nodes_from(node.set_index('title').T.to_dict().items())
graph.add_edges_from(edge[['source', 'target']].values.tolist())
print(graph.number_of_nodes(), graph.number_of_edges())

4770 6871


In [ ]:
# 역 이름이 들어간 것 이름 바꾸기 위함
titleDF = pd.merge(left=trans, right=subway, left_on='newID_승차', right_on='ARS_ID', how='left')
titleDF = titleDF[['승차_title', 'title']]
titleDF

titleDF2 = pd.merge(left=trans, right=subway, left_on='newID_하차', right_on='ARS_ID', how='left')
titleDF2 = titleDF2[['하차_title', 'title']]
titleDF2

for i in range(len(trans)):
    if (pd.isna(titleDF["title"].iloc[i])):
        trans["승차_title"].iloc[i] = titleDF["승차_title"].iloc[i]
    else:
        trans["승차_title"].iloc[i] = titleDF["title"].iloc[i]
    if (pd.isna(titleDF2["title"].iloc[i])):
        trans["하차_title"].iloc[i] = titleDF2["하차_title"].copy().iloc[i]
    else:
        trans["하차_title"].iloc[i] = titleDF2["title"].copy().iloc[i]

,하차_title,title
0,308_중계역,308_중계역
1,345_혜화역.동성중고,345_혜화역
2,345_혜화역.마로니에공원,345_혜화역
3,345_혜화역.마로니에공원,345_혜화역
4,345_혜화역.동성중고,345_혜화역
...,...,...
89863,35181_고양동삼거리,NaN
89864,31612_광탄행정복지센터,NaN
89865,31584_용미2리,NaN
89866,35605_고양현대아파트,NaN


In [ ]:
#공백제거
trans["승차_title"]=trans['승차_title'].str.replace(" ","")
trans["하차_title"]=trans['하차_title'].str.replace(" ","")

In [ ]:
# 데이터 간의 정류장 이름이 다른 경우로 인해 발생하는 오류 제거
trans.loc[trans["하차_title"]=="1037_동대문","하차_title"] = "99_동대문역"
trans.loc[trans["승차_title"]=="1037_동대문","승차_title"] = "99_동대문역"

trans.loc[trans["하차_title"]=="1037_동대문(흥인지문)","하차_title"] = "99_동대문역"
trans.loc[trans["승차_title"]=="1037_동대문(흥인지문)","승차_title"] = "99_동대문역"

trans.loc[trans["하차_title"]=="1235_동묘역앞","하차_title"] = "102_동묘앞역"
trans.loc[trans["승차_title"]=="1235_동묘역앞","승차_title"] = "102_동묘앞역"

trans.loc[trans["하차_title"]=="1236_동묘역앞","하차_title"] = "102_동묘앞역"
trans.loc[trans["승차_title"]=="1236_동묘역앞","승차_title"] = "102_동묘앞역"

# trans.loc[trans["하차_title"]=="23297_래미안.그레이튼.아파트","하차_title"] = "23297_래미안그레이튼아파트"
# trans.loc[trans["승차_title"]=="23297_래미안.그레이튼.아파트","승차_title"] = "23297_래미안그레이튼아파트"

# trans.loc[trans["하차_title"]=="23181_일지아트홀.압구정동노인복지센터","하차_title"] = "23181_일지아트홀.압구정노인복지관"
# trans.loc[trans["승차_title"]=="23181_일지아트홀.압구정동노인복지센터","승차_title"] = "23181_일지아트홀.압구정노인복지관"

trans.loc[trans["하차_title"]=="21150_보라매공원.보라매병원","하차_title"] = "21150_당곡역.보라매공원.보라매병원"
trans.loc[trans["승차_title"]=="21150_보라매공원.보라매병원","승차_title"] = "21150_당곡역.보라매공원.보라매병원"

trans.loc[trans["하차_title"]=="21151_롯데백화점관악점","하차_title"] = "21151_당곡역2번출구.롯데백화점관악점"
trans.loc[trans["승차_title"]=="21151_롯데백화점관악점","승차_title"] = "21151_당곡역2번출구.롯데백화점관악점"

trans.loc[trans["하차_title"]=="21180_관악농협신림지점앞","하차_title"] = "21180_서울대벤처타운역"
trans.loc[trans["승차_title"]=="21180_관악농협신림지점앞","승차_title"] = "21180_서울대벤처타운역"

# trans.loc[trans["하차_title"]=="56078_서안이노빌아파트","하차_title"] = "56078_서안이노빌.평촌어바인퍼스트"
# trans.loc[trans["승차_title"]=="56078_서안이노빌아파트","승차_title"] = "56078_서안이노빌.평촌어바인퍼스트"

trans.loc[trans["하차_title"]=="16235_가양도시개발4단지아파트","하차_title"] = "16235_가양4단지아파트"
trans.loc[trans["승차_title"]=="16235_가양도시개발4단지아파트","승차_title"] = "16235_가양4단지아파트"

trans.loc[trans["하차_title"]=="16237_가양도시개발5단지아파트","하차_title"] = "16237_가양5단지아파트"
trans.loc[trans["승차_title"]=="16237_가양도시개발5단지아파트","승차_title"] = "16237_가양5단지아파트"

trans.loc[trans["하차_title"]=="23437_세곡푸르지오","하차_title"] = "23437_세곡푸르지오.은곡삼거리"
trans.loc[trans["승차_title"]=="23437_세곡푸르지오","승차_title"] = "23437_세곡푸르지오.은곡삼거리"

trans.loc[trans["하차_title"]=="23439_세곡푸르지오","하차_title"] = "23439_세곡푸르지오.은곡삼거리"
trans.loc[trans["승차_title"]=="23439_세곡푸르지오","승차_title"] = "23439_세곡푸르지오.은곡삼거리"

#trans.loc[trans["하차_title"]=="36184_백석1동행정복지센터","하차_title"] = "36184_백석1동행정복지센터.유니테크빌"
#trans.loc[trans["승차_title"]=="36184_백석1동행정복지센터","승차_title"] = "36184_백석1동행정복지센터.유니테크빌"

# trans.loc[trans["하차_title"]=="38236_원가학","하차_title"] = "36236_원가학.학온동행정복지센터"
# trans.loc[trans["승차_title"]=="38236_원가학","승차_title"] = "36236_원가학.학온동행정복지센터"

# trans.loc[trans["하차_title"]=="38246_원가학","하차_title"] = "36246_원가학.학온동행정복지센터"
# trans.loc[trans["승차_title"]=="38246_원가학.","승차_title"] = "36246_원가학.학온동행정복지센터"

trans.loc[trans["하차_title"]== "36612_일산자이센트리지.일산자이3차.","하차_title"] = "36612_일산자이센트리지.일산자이더헤리티지"
trans.loc[trans["승차_title"]=="36612_일산자이센트리지.일산자이3차.","승차_title"] = "36612_일산자이센트리지.일산자이더헤리티지"

In [ ]:
# shortest path로 edge별 혼잡지수 구하기
# PathError 나는 것 존재 -> 시간차로 인한 버스 노선의 수정 존재

lane_li = list(trans["노선명"].unique())
list_tmp_df = []
for sub_lane in lane_li:

    print(sub_lane)

    sub_edge=edge[edge["LaneNo"]==sub_lane].reset_index(drop=True)
    # sb에서 weight를 계속 더해줄 것
    sub_edge["weight"]=0



    confirm1 = list(sub_edge["source"].values)
    confirm2 = list(sub_edge["target"].values)
    confirm1.extend(confirm2)
    confirm1 = list(set(confirm1))
    sub_node= node[(node["title"].isin(confirm1))].reset_index(drop=True)
    sub_node

    ###
    sourceNode = trans[trans["노선명"]==sub_lane].reset_index(drop=True)["승차_title"]
    targetNode = trans[trans["노선명"]==sub_lane].reset_index(drop=True)["하차_title"]
    _weight = trans[trans["노선명"]==sub_lane].reset_index(drop=True)["혼잡지수"]
    ####
    for j in range(len(sourceNode)):
        try:
            path = nx.shortest_path(graph, source=sourceNode[j], target=targetNode[j], weight="distance")
        except Exception as e:
            error = pd.DataFrame(trans[trans["노선명"]==sub_lane].reset_index(drop=True).iloc[j]).T
            list_tmp_df.append(error)
            print("error", e)
            continue

        #path
        for i in range(len(path)):
            if i == (len(path)-1):
                break
            sub_edge.loc[(sub_edge["LaneNo"]==sub_lane) & (sub_edge["source"]==path[i]) & (sub_edge["target"]==path[i+1]),"weight"] += _weight[j]
    if sub_lane ==lane_li[0]:
        final_edge = sub_edge
    else:
        final_edge = pd.concat([final_edge,sub_edge])

error_Df = pd.concat(list_tmp_df, ignore_index=True)

100
101
102
103
error No path between 181_서울역 and 11493_삼화상운.
error No path between 99_동대문역 and 11493_삼화상운.
error No path between 1015_종로3가.탑골공원 and 11493_삼화상운.
error No path between 1013_종로2가 and 11493_삼화상운.
error No path between 2139_우리은행종로지점 and 11493_삼화상운.
error No path between 1021_종로6가.동대문종합시장 and 11493_삼화상운.
error No path between 1017_종로4가.종묘 and 11493_삼화상운.
error No path between 2142_롯데영프라자 and 11493_삼화상운.
error No path between 8002_길음2동주민센터 and 11493_삼화상운.
error No path between 8101_숭곡초교입구 and 11493_삼화상운.
error No path between 8008_돈암사거리.성신여대입구 and 11493_삼화상운.
error No path between 8006_미아리고개.미아리예술극장 and 11493_삼화상운.
error No path between 8281_청소년문화누림센터 and 11493_삼화상운.
error No path between 8294_장곡초교 and 11493_삼화상운.
error No path between 8221_장곡초교사거리 and 11493_삼화상운.
error No path between 8186_월계2교 and 11493_삼화상운.
error No path between 8188_광운초교 and 11493_삼화상운.
error No path between 8219_꿈의숲아이파크아파트 and 11493_삼화상운.
error No path between 8242_장위래미안아파트 and 11493_삼화상운.
error No path

In [ ]:
final_edge["weight"].max()

0.964864864864858

In [ ]:
final_edge.to_csv("./혼잡지수_bus_edge.csv", index=False, encoding="cp949")

In [ ]:
edge

,LaneNo,source,target,type,weight
0,1,90_덕정역,89_덕계역,subway,0
1,1,63_남영역,272_용산역,subway,0
2,1,272_용산역,68_노량진역,subway,0
3,1,68_노량진역,84_대방역,subway,0
4,1,84_대방역,210_신길역,subway,0
...,...,...,...,...,...
13204,775,63240_교육지원청,63242_파주시청,bus,0
13205,775,63242_파주시청,63233_장안아파트,bus,0
13206,775,63233_장안아파트,63201_금촌사거리,bus,0
13207,775,63201_금촌사거리,63213_장안미래아파트,bus,0


### 혼잡지수_지하철

In [ ]:
sub_info = pd.read_csv("/content/drive/MyDrive/데이터마이닝팀플/데이터/대중교통_월간통계자료_2023.08월/지하철/역별OD/지하철_역별OD_20230831.csv",encoding="cp949")
sub_info = sub_info.iloc[:,1:6]
sub_info.head(3)

,승차_호선,승차_역,하차_호선,하차_역,총_승객수
0,우이신설선,4.19민주묘지,우이신설선,4.19민주묘지,8
1,우이신설선,4.19민주묘지,8호선,가락시장,2
2,우이신설선,4.19민주묘지,3호선,가락시장,2


In [ ]:
sub_info["승차_title"] = ""
sub_info["하차_title"] = ""

In [ ]:
# 역 괄호 제거
def delWord(station):
    if ('(' in station) and (')' in station):
        idx1 = station.index('(')
        idx2 = station.index(')')
        return station[:idx1] + station[idx2+1:]
    return station
sub_info['승차_역'] = sub_info['승차_역'].map(delWord)
sub_info['하차_역'] = sub_info['하차_역'].map(delWord)

In [ ]:
# ARS_ID와 역 명으로 고유한 ID 생성
from difflib import SequenceMatcher

sub_info = sub_info[sub_info['승차_역'] != sub_info['하차_역']].reset_index(drop=True)
caution_idx =[]
for j in range(len(sub_info)):
    print(j)
    if len(node_subway[node_subway["title"].str.contains(sub_info["승차_역"].iloc[j])]) == 0: # 서울에 한하기 때문에 노드에 존재하지 않을 수 있음
        caution_idx.append(j)
    elif len(node_subway[node_subway["title"].str.contains(sub_info["승차_역"].iloc[j])]) != 1: # 예를 들어 강남구청역과 강남역이 있을 때 승차 역이 "강남" 이면 강남역, "강남구청" 이면 강남구청으로 하기 위함
        idx_li =[]
        for i in range(len(node_subway[node_subway["title"].str.contains(sub_info["승차_역"].iloc[j])])):
            idx = SequenceMatcher(None, node_subway[node_subway["title"].str.contains(sub_info["승차_역"].iloc[j])]["name"].values[i], sub_info["승차_역"].iloc[j]).ratio()
            idx_li.append(idx)
        fi_idx = idx_li.index(max(idx_li))
        sub_info["승차_title"].iloc[j] = node_subway[node_subway["title"].str.contains(sub_info["승차_역"].iloc[j])]["title"].values[fi_idx]

    else:
        sub_info["승차_title"].iloc[j] = node_subway[node_subway["title"].str.contains(sub_info["승차_역"].iloc[j])]["title"].values[0]

    if len(node_subway[node_subway["title"].str.contains(sub_info["하차_역"].iloc[j])]) == 0:
        caution_idx.append(j)
    elif len(node_subway[node_subway["title"].str.contains(sub_info["하차_역"].iloc[j])]) !=1:
        idx_li = []
        for i in range(len(node_subway[node_subway["title"].str.contains(sub_info["하차_역"].iloc[j])])):
            idx = SequenceMatcher(None, node_subway[node_subway["title"].str.contains(sub_info["하차_역"].iloc[j])]["name"].values[i], sub_info["하차_역"].iloc[j]).ratio()
            idx_li.append(idx)
        fi_idx = idx_li.index(max(idx_li))
        sub_info["하차_title"].iloc[j] = node_subway[node_subway["title"].str.contains(sub_info["하차_역"].iloc[j])]["title"].values[fi_idx]
    else:
        sub_info["하차_title"].iloc[j] = node_subway[node_subway["title"].str.contains(sub_info["하차_역"].iloc[j])]["title"].values[0]
sub_info.head()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


<ipython-input-171-d3fa92aad510>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_info["승차_title"].iloc[j] = node_subway[node_subway["title"].str.contains(sub_info["승차_역"].iloc[j])]["title"].values[0]
<ipython-input-171-d3fa92aad510>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_info["하차_title"].iloc[j] = node_subway[node_subway["title"].str.contains(sub_info["하차_역"].iloc[j])]["title"].values[0]
<ipython-input-171-d3fa92aad510>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/us

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
1208
1209
1210
1211
1212
1213
1214
1215
1216
1217
1218
1219
1220
1221
1222
1223
1224
1225
1226
1227
1228
1229
1230
1231
1232
1233
1234
1235
1236
1237
1238
1239
1240
1241
1242
1243
1244
1245
1246
1247
1248
1249
1250
1251
1252
1253
1254
1255
1256
1257
1258
1259
1260
1261
1262
1263
1264
1265
1266
1267
1268
1269
1270
1271
1272
1273
1274
1275
1276
1277
1278
1279
1280
1281
1282
1283
1284
1285
1286
1287
1288
1289
1290
1291
1292
1293
1294
1295
1296
1297
1298
1299
1300
1301
1302
1303
1304
1305
1306
1307
1308
1309
1310
1311
1312
1313
1314
1315
1316
1317
1318
1319
1320
1321
1322
1323
1324
1325
1326
1327
1328
1329
1330
1331
1332
1333
1334
1335
1336
1337
1338
1339
1340
1341
1342
1343
1344
1345
1346
1347
1348
1349
1350
1351
1352
1353
1354
1355
1356
1357
1358
1359
1360
1361
1362
1363
1364
1365
1366
1367
1368
1369
1370
1371
1372
1373
1374
1375
1376
1377
1378
1379
1380
1381
1382
1383
1384
1385
1386
1387
1388
1389
1390
1391
1392
1393
1394
1395
1396
1397
1398
1399
1400

<ipython-input-171-d3fa92aad510>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_info["승차_title"].iloc[j] = node_subway[node_subway["title"].str.contains(sub_info["승차_역"].iloc[j])]["title"].values[fi_idx]


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
267266
267267
267268
267269
267270
267271
267272
267273
267274
267275
267276
267277
267278
267279
267280
267281
267282
267283
267284
267285
267286
267287
267288
267289
267290
267291
267292
267293
267294
267295
267296
267297
267298
267299
267300
267301
267302
267303
267304
267305
267306
267307
267308
267309
267310
267311
267312
267313
267314
267315
267316
267317
267318
267319
267320
267321
267322
267323
267324
267325
267326
267327
267328
267329
267330
267331
267332
267333
267334
267335
267336
267337
267338
267339
267340
267341
267342
267343
267344
267345
267346
267347
267348
267349
267350
267351
267352
267353
267354
267355
267356
267357
267358
267359
267360
267361
267362
267363
267364
267365
267366
267367
267368
267369
267370
267371
267372
267373
267374
267375
267376
267377
267378
267379
267380
267381
267382
267383
267384
267385
267386
267387
267388
267389
267390
267391
267392
267393
267394
267395
267396
267397
267398
267399
267400
267401
267402
26740

,승차_호선,승차_역,하차_호선,하차_역,총_승객수,승차_title,하차_title
0,우이신설선,4.19민주묘지,8호선,가락시장,2,1_4.19민주묘지역,3_가락시장역
1,우이신설선,4.19민주묘지,3호선,가락시장,2,1_4.19민주묘지역,3_가락시장역
2,우이신설선,4.19민주묘지,1호선,가산디지털단지,1,1_4.19민주묘지역,4_가산디지털단지역
3,우이신설선,4.19민주묘지,7호선,가산디지털단지,9,1_4.19민주묘지역,4_가산디지털단지역
4,우이신설선,4.19민주묘지,9호선,가양,1,1_4.19민주묘지역,5_가양역


In [ ]:
sub_info.to_csv('./중간_sub_info.csv', index=None, encoding='cp949')

In [ ]:
sub_info = pd.read_csv('./중간_sub_info.csv', encoding='cp949')

In [ ]:
sub_info

,승차_호선,승차_역,하차_호선,하차_역,총_승객수,승차_title,하차_title
0,우이신설선,4.19민주묘지,8호선,가락시장,2,1_4.19민주묘지역,3_가락시장역
1,우이신설선,4.19민주묘지,3호선,가락시장,2,1_4.19민주묘지역,3_가락시장역
2,우이신설선,4.19민주묘지,1호선,가산디지털단지,1,1_4.19민주묘지역,4_가산디지털단지역
3,우이신설선,4.19민주묘지,7호선,가산디지털단지,9,1_4.19민주묘지역,4_가산디지털단지역
4,우이신설선,4.19민주묘지,9호선,가양,1,1_4.19민주묘지역,5_가양역
...,...,...,...,...,...,...,...
272261,의정부선,흥선,2호선,홍대입구,2,NaN,346_홍대입구역
272262,의정부선,흥선,3호선,홍제,2,NaN,347_홍제역
272263,의정부선,흥선,경의중앙선,회기,1,NaN,353_회기역
272264,의정부선,흥선,의정부선,회룡,4,NaN,354_회룡역


In [ ]:
# 서울에 한한 지하철 데이터만 추
sub_info = sub_info[(sub_info["승차_title"]!="") & (sub_info["하차_title"]!="")].reset_index(drop=True)
sub_info = sub_info[(~pd.isna(sub_info["승차_title"])) & (~pd.isna(sub_info["하차_title"]))].reset_index(drop=True)

In [ ]:
# 지하철 노선 확인
print(sorted(list(sub_info[(sub_info["승차_title"]!="") & (sub_info["하차_title"]!="")]["승차_호선"].unique())))
print(sorted(list(sub_info[(sub_info["승차_title"]!="") & (sub_info["하차_title"]!="")]["하차_호선"].unique())))

['1호선', '2호선', '3호선', '4호선', '5호선', '6호선', '7호선', '7호선(인천)', '8호선', '9호선', '경의중앙선', '경춘선', '공항철도1호선', '김포골드라인', '분당선', '서해선', '신림선', '신분당선', '신분당선(연장2)', '우이신설선', '의정부선', '인천1호선', '인천2호선']
['1호선', '2호선', '3호선', '4호선', '5호선', '6호선', '7호선', '7호선(인천)', '8호선', '9호선', '경의중앙선', '경춘선', '공항철도1호선', '김포골드라인', '분당선', '서해선', '신림선', '신분당선', '신분당선(연장2)', '우이신설선', '의정부선', '인천1호선', '인천2호선']


In [ ]:
sub_info.head(3)

,승차_호선,승차_역,하차_호선,하차_역,총_승객수,승차_title,하차_title
0,우이신설선,4.19민주묘지,8호선,가락시장,2,1_4.19민주묘지역,3_가락시장역
1,우이신설선,4.19민주묘지,3호선,가락시장,2,1_4.19민주묘지역,3_가락시장역
2,우이신설선,4.19민주묘지,1호선,가산디지털단지,1,1_4.19민주묘지역,4_가산디지털단지역


In [ ]:
sourceNode = sub_info["승차_title"]
targetNode = sub_info["하차_title"]
_weight = sub_info["총_승객수"]
sub_edge = edge[edge["type"]=="subway"]

for j in range(len(sourceNode)):
    try:
        path = nx.shortest_path(graph, source=sourceNode[j], target=targetNode[j], weight="distance")
    except Exception as e:
        #error = pd.DataFrame(trans[trans["노선명"]==sub_lane].reset_index(drop=True).iloc[j]).T
        #list_tmp_df.append(error)
        print("error", e)
        continue

    #path
    for i in range(len(path)):
        if i == (len(path)-1):
            break
        sub_edge.loc[(sub_edge["source"]==path[i]) & (sub_edge["target"]==path[i+1]),"weight"] += _weight[j]
    print(j)
# if j==0:
#     final_edge2 = sub_edge
# else:
#     final_edge2 = pd.concat([final_edge2,sub_edge])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
156000
156001
156002
156003
156004
156005
156006
156007
156008
156009
156010
156011
156012
156013
156014
156015
156016
156017
156018
156019
156020
156021
156022
156023
156024
156025
156026
156027
156028
156029
156030
156031
156032
156033
156034
156035
156036
156037
156038
156039
156040
156041
156042
156043
156044
156045
156046
156047
156048
156049
156050
156051
156052
156053
156054
156055
156056
156057
156058
156059
156060
156061
156062
156063
156064
156065
156066
156067
156068
156069
156070
156071
156072
156073
156074
156075
156076
156077
156078
156079
156080
156081
156082
156083
156084
156085
156086
156087
156088
156089
156090
156091
156092
156093
156094
156095
156096
156097
156098
156099
156100
156101
156102
156103
156104
156105
156106
156107
156108
156109
156110
156111
156112
156113
156114
156115
156116
156117
156118
156119
156120
156121
156122
156123
156124
156125
156126
156127
156128
156129
156130
156131
156132
156133
156134
156135
156136
15613

In [ ]:
sub_edge["LaneNo"].unique()

array([1, 2, 3, 4, 5, 6, 7, 8, 9, 100, 101, 104, 108, 109, 113],
      dtype=object)

In [ ]:
# 혼잡지수를 구하기 위한 과정
sub_edge["편성당칸수"]=0
sub_edge.loc[sub_edge["LaneNo"]==113,"편성당칸수"]=2
sub_edge.loc[sub_edge["LaneNo"]==100,"편성당칸수"]=6
sub_edge.loc[sub_edge["LaneNo"]==101,"편성당칸수"]=6
sub_edge.loc[sub_edge["LaneNo"]==104,"편성당칸수"]=8
sub_edge.loc[sub_edge["LaneNo"]==108,"편성당칸수"]=8
sub_edge.loc[sub_edge["LaneNo"]==109,"편성당칸수"]=6

sub_edge.loc[sub_edge["LaneNo"]==1,"편성당칸수"]=10
sub_edge.loc[sub_edge["LaneNo"]==2,"편성당칸수"]=10
sub_edge.loc[sub_edge["LaneNo"]==3,"편성당칸수"]=10
sub_edge.loc[sub_edge["LaneNo"]==4,"편성당칸수"]=10
sub_edge.loc[sub_edge["LaneNo"]==5,"편성당칸수"]=8
sub_edge.loc[sub_edge["LaneNo"]==6,"편성당칸수"]=8
sub_edge.loc[sub_edge["LaneNo"]==7,"편성당칸수"]=8
sub_edge.loc[sub_edge["LaneNo"]==8,"편성당칸수"]=6
sub_edge.loc[sub_edge["LaneNo"]==9,"편성당칸수"]=6

sub_edge["운행횟수"]=0
sub_edge.loc[sub_edge["LaneNo"]==113,"운행횟수"]=357
sub_edge.loc[sub_edge["LaneNo"]==100,"운행횟수"]=352
sub_edge.loc[sub_edge["LaneNo"]==101,"운행횟수"]=179
sub_edge.loc[sub_edge["LaneNo"]==104,"운행횟수"]=109
sub_edge.loc[sub_edge["LaneNo"]==108,"운행횟수"]=137
sub_edge.loc[sub_edge["LaneNo"]==109,"운행횟수"]=327

sub_edge.loc[sub_edge["LaneNo"]==1,"운행횟수"]=589
sub_edge.loc[sub_edge["LaneNo"]==2,"운행횟수"]=528
sub_edge.loc[sub_edge["LaneNo"]==3,"운행횟수"]=407
sub_edge.loc[sub_edge["LaneNo"]==4,"운행횟수"]=476
sub_edge.loc[sub_edge["LaneNo"]==5,"운행횟수"]=428
sub_edge.loc[sub_edge["LaneNo"]==6,"운행횟수"]=340
sub_edge.loc[sub_edge["LaneNo"]==7,"운행횟수"]=393
sub_edge.loc[sub_edge["LaneNo"]==8,"운행횟수"]=292
sub_edge.loc[sub_edge["LaneNo"]==9,"운행횟수"]=110
sub_edge["혼잡지수"]=0
for i in range(len(sub_edge)):
    sub_edge["혼잡지수"].iloc[i] = sub_edge["weight"].iloc[i]/(sub_edge["운행횟수"].iloc[i]*160*(sub_edge["편성당칸수"].iloc[i]))


<ipython-input-76-d2f20c015058>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_edge["편성당칸수"]=0
<ipython-input-76-d2f20c015058>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_edge["운행횟수"]=0
<ipython-input-76-d2f20c015058>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

In [ ]:
sub_edge.to_csv("./혼잡지수_subway_edge.csv", index=False, encoding="cp949")

In [ ]:
sub_edge1= sub_edge.iloc[:,[0,1,2,3,7]]
sub_edge1 = sub_edge1.rename(columns={'혼잡지수':'weight'})
edge = pd.concat([sub_edge1,final_edge])
edge.reset_index(drop=True,inplace=True)
edge

,LaneNo,source,target,type,weight
0,1,90_덕정역,89_덕계역,subway,0.002701
1,1,63_남영역,272_용산역,subway,0.072168
2,1,272_용산역,68_노량진역,subway,0.053546
3,1,68_노량진역,84_대방역,subway,0.024140
4,1,84_대방역,210_신길역,subway,0.039876
...,...,...,...,...,...
7166,774,12019_동명여고.천주교불광동성당,256_연신내역,bus,0.019185
7167,774,256_연신내역,12015_세명컴퓨터고등학교,bus,0.095927
7168,774,12015_세명컴퓨터고등학교,12013_박석고개.신도고등학교,bus,0.104782
7169,774,12013_박석고개.신도고등학교,46_구파발역,bus,0.095927


In [ ]:
len(node)

4770

In [ ]:
len(edge)

7171

In [ ]:
# 노선 지워도 혼자서 따로 노는 노드 존재
confirm1 = list(edge["source"].values)
confirm2 = list(edge["target"].values)
confirm1.extend(confirm2)
confirm1=list(set(confirm1))
len(set(confirm1))

3262

In [ ]:
# 혼자 따로 노는 노드 삭제
node =node[(node["title"].isin(confirm1))].reset_index(drop=True)
node

,title,ARS_ID,name,X,Y,type
0,1_4.19민주묘지역,1,4.19민주묘지역,37.649527,127.013689,subway
1,2_가능역,2,가능역,37.748393,127.044275,subway
2,3_가락시장역,3,가락시장역,37.493092,127.118258,subway
3,4_가산디지털단지역,4,가산디지털단지역,37.480389,126.882644,subway
4,5_가양역,5,가양역,37.561429,126.854402,subway
...,...,...,...,...,...,...
3257,68233_장냇말,68233,장냇말,37.726196,126.976828,bus
3258,68234_장골부대앞,68234,장골부대앞,37.731134,126.977772,bus
3259,68235_부곡1리마을회관,68235,부곡1리마을회관,37.736108,126.977397,bus
3260,68236_송추초등학교.송추검문소,68236,송추초등학교.송추검문소,37.716477,126.971601,bus


In [10]:
graph = nx.DiGraph()
graph.add_nodes_from(node.set_index('title').T.to_dict().items())
graph.add_edges_from(edge[['source', 'target']].values.tolist())
print(graph.number_of_nodes(), graph.number_of_edges())

3262 4642


In [ ]:
# 파일 저장
node.to_csv('./혼잡지수_node.csv', index=None, encoding='cp949')
edge.to_csv('./혼잡지수_edge.csv', index=None, encoding='cp949')
nx.write_graphml_lxml(graph, '혼잡지수_graph.graphml')

In [4]:
node = pd.read_csv('./혼잡지수_node.csv', encoding='cp949')
edges = pd.read_csv('./혼잡지수_edge.csv', encoding='cp949')

In [6]:
edge_list = edges.loc[:, ('source', 'target')].values.tolist()
edge_list[:5]

[['90_덕정역', '89_덕계역'],
 ['63_남영역', '272_용산역'],
 ['272_용산역', '68_노량진역'],
 ['68_노량진역', '84_대방역'],
 ['84_대방역', '210_신길역']]

In [8]:
# Add weight
#-----your code-----#
edges_weight=[]
for i in range(len(edge_list)):
    #edges_weight.append([edges.source[i],edges.target[i],{"weight":float(edges.total_support[i])}])
    edges_weight.append([edges.source[i],edges.target[i],edges.weight[i]])

#-----valid-----#
edges_weight

[['90_덕정역', '89_덕계역', 0.0027005517826825],
 ['63_남영역', '272_용산역', 0.072167869269949],
 ['272_용산역', '68_노량진역', 0.0535462648556876],
 ['68_노량진역', '84_대방역', 0.024140492359932],
 ['84_대방역', '210_신길역', 0.0398758488964346],
 ['210_신길역', '260_영등포역', 0.0592816213921901],
 ['260_영등포역', '217_신도림역', 0.1328162139219015],
 ['217_신도림역', '40_구로역', 0.1452408743633276],
 ['40_구로역', '45_구일역', 0.0827228353140916],
 ['45_구일역', '16_개봉역', 0.0792720713073005],
 ['262_오류동역', '265_온수역', 0.0608828522920203],
 ['265_온수역', '253_역곡역', 0.0500541171477079],
 ['181_서울역', '63_남영역', 0.0728013582342954],
 ['253_역곡역', '193_소사역', 0.0350891341256366],
 ['150_부천역', '310_중동역', 0.0153087860780984],
 ['310_중동역', '196_송내역', 0.0113338285229202],
 ['40_구로역', '4_가산디지털단지역', 0.0603809422750424],
 ['4_가산디지털단지역', '97_독산역', 0.05],
 ['97_독산역', '52_금천구청역', 0.0401994906621392],
 ['52_금천구청역', '184_석수역', 0.0344662563667232],
 ['184_석수역', '32_관악역', 0.0310918930390492],
 ['32_관악역', '236_안양역', 0.0282735568760611],
 ['236_안양역', '126_명학역', 0.021

In [ ]:
graph = nx.DiGraph()
graph.add_nodes_from(node.set_index('title').T.to_dict().items())
graph.add_edges_from(edge[['source', 'target']].values.tolist())
print(graph.number_of_nodes(), graph.number_of_edges())

In [11]:
graph.add_weighted_edges_from(edges_weight)

In [12]:
for i, edata in enumerate(graph.edges(data=True)):
    print(i, edata)
    break

0 ('1_4.19민주묘지역', '194_솔밭공원역', {'weight': 0.0266369047619047})


In [13]:
nx.write_graphml_lxml(graph, "혼잡지수_weight_graph.graphml")